In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Import data as a dataframe

freestyle = pd.read_csv('speedrun_data.csv')

#Reorganize indexes
# speed_df.set_index(['Star Rating','Monster Name'],inplace=True)

freestyle.head()

,Star Rating,Monster Name,Quest,Runner,Time,Weapon,Platform,Ruleset
0,6,Alatreon,The Evening Star,DEVA,"01'55""11",Heavy Bowgun,PC,Freestyle
1,6,Alatreon,The Evening Star,DEVA,"02'05""13",Heavy Bowgun,PC,Freestyle
2,6,Alatreon,The Evening Star,Jackdaw갈까마귀,"02'24""56",Heavy Bowgun,PC,Freestyle
3,6,Alatreon,The Evening Star,Jackdaw갈까마귀,"02'30""96",Heavy Bowgun,PC,Freestyle
4,6,Alatreon,Dawn of the Death Star,Jackdaw갈까마귀,"02'44""40",Heavy Bowgun,PC,Freestyle


In [3]:
# Filter out 'repeat runs'
# 'Repeat run': run on the same monster, same quest, by the same runner with the same weapon, but different times.
# Keep only the fastest run

freestyle = freestyle.drop_duplicates(['Monster Name','Quest','Runner','Weapon'],keep='first') #Already sorted by time,
                                                                                               #first time = fastest!


In [4]:
#Function to convert the time field to seconds
def to_seconds(time_string):
    
    #Get minutes, seconds and centi seconds from string
    minutes = int(time_string[0:2])
    seconds = int(time_string[3:5])
    cents = int(time_string[6:])
    
    #Return total in seconds
    return 60*minutes+seconds+cents/100
    
#Add Time (Seconds) column
freestyle.insert(5, 'Time (Seconds)', freestyle['Time'].apply(to_seconds))

In [5]:
#Convert Star Rating to string (discrete values!)
freestyle['Star Rating']=freestyle['Star Rating'].apply(str)

In [6]:
#Abbreviate weapon names

weapon_dict = {'Great Sword':'GS',
               'Long Sword':'LS',
               'Sword And Shield':'SnS',
               'Dual Blades':'DB',
               'Lance':'Lance',
               'Gunlance':'GL',
               'Hammer':'Hammer',
               'Hunting Horn':'HH',
               'Switch Axe':'SA',
               'Charge Blade':'CB',
               'Insect Glaive':'IG',
               'Heavy Bowgun':'HBG',
               'Light Bowgun':'LBG',
               'Bow':'Bow'    
               }

#Apply dict to weapons column
freestyle['Weapon']=freestyle['Weapon'].apply(lambda x: weapon_dict[x])

In [7]:
# Sort dataframe by, in order: Star Rating, Monster Name, Quest, and Time
freestyle = freestyle.sort_values(['Star Rating','Monster Name','Quest','Time (Seconds)'],ascending=[False,True,True,True])

In [8]:
#Fix indexes
freestyle = freestyle.reset_index()

In [9]:
# Retrieve all monsters,quests and weapons to make rankings the rankings
monster_list = freestyle['Monster Name'].unique()
quest_list = freestyle['Quest'].unique()
weapon_list = freestyle['Weapon'].unique()

In [10]:
#Add 4 columns for the 4 rankings:

#Quest/General: rank all runs for the same QUEST consecutively
#Quest/Weapon: rank all runs for the same QUEST by weapon type

#Monster/General: rank all runs for the same monster consecutively
#Monster/Weapon: rank all runs for the same monster by weapon type

rank_list = ['Quest/General', 'Quest/Weapon','Monster/General', 'Monster/Weapon',]
freestyle = pd.concat([freestyle,pd.DataFrame(columns=rank_list)],axis=1)
freestyle

,index,Star Rating,Monster Name,Quest,Runner,Time,Time (Seconds),Weapon,Platform,Ruleset,Quest/General,Quest/Weapon,Monster/General,Monster/Weapon
0,4,6,Alatreon,Dawn of the Death Star,Jackdaw갈까마귀,"02'44""40",164.40,HBG,PC,Freestyle,NaN,NaN,NaN,NaN
1,7,6,Alatreon,Dawn of the Death Star,Relial,"02'58""53",178.53,HBG,PC,Freestyle,NaN,NaN,NaN,NaN
2,13,6,Alatreon,Dawn of the Death Star,Randel Daora,"03'26""71",206.71,HBG,PC,Freestyle,NaN,NaN,NaN,NaN
3,17,6,Alatreon,Dawn of the Death Star,HK_MH_OLAY,"03'41""38",221.38,GS,PC,Freestyle,NaN,NaN,NaN,NaN
4,21,6,Alatreon,Dawn of the Death Star,ブロントさん,"03'52""16",232.16,SnS,PC,Freestyle,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5729,8225,1,Tzitzi-Ya-Ku,All the Wrong Signals,Lumini,"03'16""23",196.23,HH,PS4,Freestyle,NaN,NaN,NaN,NaN
5730,8226,1,Tzitzi-Ya-Ku,All the Wrong Signals,CBonduMiel,"03'37""48",217.48,GS,PC,Freestyle,NaN,NaN,NaN,NaN
5731,8227,1,Tzitzi-Ya-Ku,All the Wrong Signals,shakybake,"04'12""08",252.08,SnS,PC,Freestyle,NaN,NaN,NaN,NaN
5732,8228,1,Tzitzi-Ya-Ku,All the Wrong Signals,Unsafekibble,"04'57""73",297.73,GS,PS4,Freestyle,NaN,NaN,NaN,NaN


In [70]:
#Rank by ???/General
# ??? can be either Quest or Monster Name
def get_rank_general(df,rank_by_column='Quest',ranked_column='Quest/General'):
    """
    Ranks speedruns by the Quest/General criteria: rank all runs for the same QUEST consecutively
    df: dataframe with speedrun data to rank
    quest_name: name of quest to rank
    """
    
    #Number of rows
    rows = df.shape[0]
    
    #Variable to remember current rank
    current_rank=0;
    
    for index, row in df.iterrows():
        
        #Get the current entry to rank by and the previous one
        current_entry = df.loc[(df.index==index),rank_by_column].to_list()[0] 
        try:
            previous_entry = df.loc[(df.index==index-1),rank_by_column].to_list()[0] 
        except IndexError:
            previous_entry = ''
            
        if (index == 0) or (current_entry != previous_entry): #First row of a new quest/monster
            df.loc[(df.index==0),ranked_column] = 1
            current_rank = 2
        else:
            df.loc[(df.index==index),ranked_column] = current_rank
            current_rank += 1
            
    return df[ranked_column]

In [71]:
#Get the two general ranks done
freestyle['Quest/General'] = get_rank_general(freestyle,'Quest','Quest/General')
freestyle['Monster/General'] = get_rank_general(freestyle,'Monster Name','Monster/General')

,index,Star Rating,Monster Name,Quest,Runner,Time,Time (Seconds),Weapon,Platform,Ruleset,Quest/General,Quest/Weapon,Monster/General,Monster/Weapon
0,4,6,Alatreon,Dawn of the Death Star,Jackdaw갈까마귀,"02'44""40",164.40,HBG,PC,Freestyle,1,NaN,1,NaN
1,7,6,Alatreon,Dawn of the Death Star,Relial,"02'58""53",178.53,HBG,PC,Freestyle,2,NaN,2,NaN
2,13,6,Alatreon,Dawn of the Death Star,Randel Daora,"03'26""71",206.71,HBG,PC,Freestyle,3,NaN,3,NaN
3,17,6,Alatreon,Dawn of the Death Star,HK_MH_OLAY,"03'41""38",221.38,GS,PC,Freestyle,4,NaN,4,NaN
4,21,6,Alatreon,Dawn of the Death Star,ブロントさん,"03'52""16",232.16,SnS,PC,Freestyle,5,NaN,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,478,6,Arch-tempered Velkhana,The Place Where Winter Sleeps,대전설GLGD,"04'47""75",287.75,DB,PC,Freestyle,12,NaN,12,NaN
296,479,6,Arch-tempered Velkhana,The Place Where Winter Sleeps,Nito,"04'53""86",293.86,LS,PC,Freestyle,13,NaN,13,NaN
297,480,6,Arch-tempered Velkhana,The Place Where Winter Sleeps,Cee,"04'55""35",295.35,SnS,PC,Freestyle,14,NaN,14,NaN
298,481,6,Arch-tempered Velkhana,The Place Where Winter Sleeps,Ryo,"04'57""31",297.31,GL,PS4,Freestyle,15,NaN,15,NaN


In [72]:
freestyle.head(200)

,index,Star Rating,Monster Name,Quest,Runner,Time,Time (Seconds),Weapon,Platform,Ruleset,Quest/General,Quest/Weapon,Monster/General,Monster/Weapon
0,4,6,Alatreon,Dawn of the Death Star,Jackdaw갈까마귀,"02'44""40",164.40,HBG,PC,Freestyle,1,NaN,1,NaN
1,7,6,Alatreon,Dawn of the Death Star,Relial,"02'58""53",178.53,HBG,PC,Freestyle,2,NaN,2,NaN
2,13,6,Alatreon,Dawn of the Death Star,Randel Daora,"03'26""71",206.71,HBG,PC,Freestyle,3,NaN,3,NaN
3,17,6,Alatreon,Dawn of the Death Star,HK_MH_OLAY,"03'41""38",221.38,GS,PC,Freestyle,4,NaN,4,NaN
4,21,6,Alatreon,Dawn of the Death Star,ブロントさん,"03'52""16",232.16,SnS,PC,Freestyle,5,NaN,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,268,6,Alatreon,The Evening Star,LVIS-Gaming,"07'43""86",463.86,CB,PC,Freestyle,63,NaN,196,NaN
196,273,6,Alatreon,The Evening Star,KCGemini,"07'49""51",469.51,Lance,PS4,TA Rules,64,NaN,197,NaN
197,274,6,Alatreon,The Evening Star,Benji-boi,"07'50""25",470.25,SA,PS4,TA Rules,65,NaN,198,NaN
198,276,6,Alatreon,The Evening Star,Fataripper,"07'50""88",470.88,DB,PS4,Freestyle,66,NaN,199,NaN


In [ ]:
# Separate TA runs. Note that Freestyle runs also encompass TA runs (ie a TA run is also a Freestyle run, but a Freestyle run
# not be a TA run)

ta = freestyle[freestyle['Ruleset']=='TA Rules'].copy()
ta = ta.drop('Ruleset',axis=1) #Drop Ruleset column as it is redundant

In [ ]:
#Very early analysis: just get average clear time by weapons

avg_freestyle = freestyle.groupby('Weapon').mean().reset_index()
avg_ta = ta.groupby('Weapon').mean().reset_index()

#Sort
avg_freestyle = avg_freestyle.sort_values('Time (Seconds)',ascending=True)
avg_ta = avg_ta.sort_values('Time (Seconds)',ascending=True)

In [ ]:
#Plot average freestyle times
plt.figure(figsize=(16,7))
sns.barplot(x='Weapon',y='Time (Seconds)',data=avg_freestyle)

In [ ]:
#Plot average TA times
plt.figure(figsize=(16,7))
sns.barplot(x='Weapon',y='Time (Seconds)',data=avg_ta)